 # SEGMENTING AND CLUSTERING NEIGHBOURHOODS IN TORONTO

## 1. Data Wrangling

In [38]:
# First we have to install beutifulsoup for scraping the website
#!conda install -c anaconda beautifulsoup4 --yes

In [37]:
# Install html parser (lxml or html5lib)
#!conda install -c anaconda lxml --yes

In [7]:
# imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [8]:
# Assign the source (Wikipedia)
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

# locate the table 
table = soup.find('table', class_='wikitable')

headers=list()
# retrieve the headers
for row in table.findAll("th"):
    headers.append(row.text.replace('\n',''))

#Create the dataframe
df=pd.DataFrame(columns=headers)

# append the data
rows=list()
for tbody in table.findAll("tbody"):
    for tr in tbody.findAll("tr"):
        for td in tr.findAll("td"):
            rows.append(td.text.replace('\n',''))
        
        if len(rows)>0:
            df = df.append({headers[0]: rows[0], headers[1]: rows[1], headers[2]: rows[2]}, ignore_index=True, sort=False)
        rows.clear()

# check the df        
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# Group by aggregating the neighbourhoods 
aggregations = {
    'Neighbourhood': lambda x:', '.join(x)
}
df = df.groupby(['Postcode', 'Borough']).agg(aggregations)
# reset index
df = df.reset_index()

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [10]:
# Delete not assigned boroughs
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df = df.sort_values(by=['Postcode'])

df.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [11]:
#Let's check if there are some not assigned values in neighbourhood column
df.loc[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
120,M7A,Queen's Park,Not assigned


In [12]:
# Copy values from borough column to those not assigned values on neighbourhood column
df.Neighbourhood = df.Borough.where(df.Neighbourhood == 'Not assigned', df.Neighbourhood)

In [13]:
# Let's check again if there are some not assigned values in neighbourhood column
df.loc[df['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood


In [14]:
df.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [15]:
df.shape

(103, 3)

## 2. Locate Neighbourhoods

In [36]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### The above code takes too long to finish (API?) so I will use the csv instead

In [19]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [20]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Change Postal code column name
df_geo.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
# Merge both dataframes
df = df.merge(df_geo, on='Postcode', how='inner')

df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Explore Neighborhoods in Toronto

In [23]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [25]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
Toronto_location = geolocator.geocode(address)
Toronto_latitude = Toronto_location.latitude
Toronto_longitude = Toronto_location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(Toronto_latitude, Toronto_longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [26]:
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [27]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Recreate the map only with the Boroughs that contains 'Toronto'

In [28]:
df_tor = df[df['Borough'].str.contains('Toronto')]

df_tor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [29]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [30]:
print('The new dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_tor['Borough'].unique()),
        df.shape[0]
    )
)

The new dataframe has 4 boroughs and 103 neighborhoods.


#### Time to Cluster Neighborhoods

In [31]:
# set number of clusters
kclusters = 4

toronto_clustering = df_tor.drop(['Postcode','Neighbourhood','Borough'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 2, 2, 2, 2, 2])

In [32]:
# add clustering labels to the original df
df_tor.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
df_tor.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,0,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,0,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,0,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Let's display the final map with the Boroughs clustered by colours

In [34]:
# create map
map_clusters = folium.Map(location=[Toronto_latitude, Toronto_longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighbourhood'], df_tor['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters